In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
path = "~/Downloads/air.mon.mean.nc"

ds = xr.open_dataset(path)

In [3]:
y = ds['air']
air = y.loc['1979-01-01':'1999-12-01']
air = air.transpose('level', 'lat', 'lon', 'time')
air

<xarray.DataArray 'air' (level: 17, lat: 73, lon: 144, time: 252)>
[45033408 values with dtype=float32]
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1999-12-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.40775 315.97388]
    units:                 degK
    precision:             2
    GRIB_id:               11
    GRIB_name:             TMP
    var_desc:              Air temperature
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Pressure Levels
    statistic:             Individual Obs
    parent_stat:           Othe

In [4]:
def clmMonLLLT(x):
    
    # Calculate the sizes of time dimension
    len_of_dim = x.sizes
    time_size = len_of_dim[x.dims[3]]
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((time_size % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
    
    # Get the sizes of latitude and longitude dimension
    lev_size = len_of_dim[x.dims[0]]
    lat_size = len_of_dim[x.dims[1]]
    lon_size = len_of_dim[x.dims[2]]
    
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[3]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    # Compute 12 months average
    ave_month = x.groupby(time_month).mean(time)
    
    # Copy and update the attributes
    ave_month.attrs = x.attrs
    ave_month = ave_month.rename("aveMonth")
    ave_month.attrs['time_op_ncl'] = "Climatology: " + str(int(time_size/no_of_months)) + " years"
    ave_month.attrs['info'] = "function clmMonTLL"
    
    # Return the results
    return ave_month

In [5]:
xAve = clmMonLLLT(air)

In [6]:
xAve

<xarray.DataArray 'aveMonth' (level: 17, lat: 73, lon: 144, month: 12)>
array([[[[245.98235, ..., 247.39333],
         ...,
         [245.98235, ..., 247.39333]],

        ...,

        [[266.21616, ..., 266.2981 ],
         ...,
         [266.21616, ..., 266.2981 ]]],


       ...,


       [[[212.21666, ..., 205.0924 ],
         ...,
         [212.21666, ..., 205.0924 ]],

        ...,

        [[249.37144, ..., 252.49666],
         ...,
         [249.37144, ..., 252.49666]]]], dtype=float32)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.4

In [7]:
def calcMonAnomLLLT(x, xAve):
    # x is the dataarray and xAve is the monthly averages array of x
    
    len_of_dim = x.sizes
    no_of_time = len_of_dim[x.dims[3]] # no_of_time = Size of time dimension
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((no_of_time % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[3]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    
    # Calculate anomalies by subtracting monthly means from actual dataarray
    xAnom = x.groupby(time_month) - xAve
        
    # Copy and update the attributes
    xAnom.attrs = x.attrs
    xAnom.attrs['anomaly_op_ncl'] = "Anomalies from Annual Cycle: calcMonAnomLLLT"
    
    # Drop the extra month co-ordinate from xAnom DataArray
    xAnom = xAnom.drop('month')
    xAnom = xAnom.rename('xAnom')
    
    # Return the new dataarray
    return xAnom

In [8]:
result = calcMonAnomLLLT(air, xAve)
result

<xarray.DataArray 'xAnom' (level: 17, lat: 73, lon: 144, time: 252)>
array([[[[-0.732346, ...,  1.986679],
         ...,
         [-0.732346, ...,  1.986679]],

        ...,

        [[ 1.453827, ..., -2.778076],
         ...,
         [ 1.453827, ..., -2.778076]]],


       ...,


       [[[-2.21666 , ..., -6.952393],
         ...,
         [-2.21666 , ..., -6.952393]],

        ...,

        [[-4.221451, ...,  3.903336],
         ...,
         [-4.221451, ...,  3.903336]]]], dtype=float32)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 1999-12-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range: 

In [9]:
result[14,14,14,0:60:5]

<xarray.DataArray 'xAnom' (time: 12)>
array([ 2.718094, -0.133316, -1.385254,  2.170013,  0.367142,  4.315247,
       -0.619553, -2.814774,  1.546646,  1.665253, -0.588089,  0.71666 ],
      dtype=float32)
Coordinates:
    level    float32 30.0
    lat      float32 55.0
    lon      float32 35.0
  * time     (time) datetime64[ns] 1979-01-01 1979-06-01 ... 1983-08-01
Attributes:
    long_name:             Monthly Air Temperature on Pressure Levels
    valid_range:           [-32765 -10260]
    unpacked_valid_range:  [137.5 362.5]
    actual_range:          [179.40775 315.97388]
    units:                 degK
    precision:             2
    GRIB_id:               11
    GRIB_name:             TMP
    var_desc:              Air temperature
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Pressure Levels
    statistic:             Individual Obs
    parent_stat:           Other
    standard_name:         air_temperature
    cell_